In [6]:
import sys
sys.path.append('/data/joramvandriel/ai-monitoring-streamlit-dashboard/')
from drift_dataset import DriftDataset
from pathlib import Path
import numpy as np
import pandas as pd
from IPython import embed

In [7]:
binnenvaart_pad = Path.home()/'share'/'Binnenvaart'/'Monitoring'

In [8]:
def mark_datasets(df: pd.DataFrame):
    """Post processing function to add markings for labelled and unlabelled samples in each dataset"""
    
    def mark_labels(df: pd.DataFrame):
        """To be applied over groups, marking the datasets"""
        df = df.copy()
        [idx] = df.dataset_index.unique()
        if idx == 0:
            train_label = 'train'
            oos_label = 'oos'
        else:
            train_label = f'labelled_{idx}'
            oos_label = f'unlabelled_{idx}'

        begindatum = max(df.Begindatum)
        mask = df.Begindatum < begindatum
        df['dataset_markering'] = None
        df.loc[mask, 'dataset_markering'] = train_label
        df.loc[~mask, 'dataset_markering'] = oos_label
        
        return df
    
    df = df.copy()
    df = df.groupby(['dataset_index']).apply(mark_labels)
    df = df.reset_index(drop=True)
    
    return df

In [9]:
print('Post processing data...')
# features uit top 10 Gini halen
gini = pd.read_csv(binnenvaart_pad/'meta'/'meanDecreaseGini.csv')
features = gini.Variable[:10].tolist() + ['Begindatum', 'dataset_markering'] # laatste wordt toegevoegd door mark_datasets()
dd = DriftDataset(binnenvaart_pad/'data', post_process=mark_datasets)
dd.keep_columns(features) 

Post processing data...


In [10]:
dd.dataset.head

<bound method NDFrame.head of        insp_weekdag dataset_markering  laatstAlgemeen  Bouwjaar  Begindatum  \
0               Fri             train            3680      1965  2015-05-22   
1               Fri             train            3680      1965  2015-05-22   
2               Wed             train            3680      1965  2017-09-27   
3               Thu             train            3680      1965  2018-01-11   
4               Wed             train            3680      1965  2016-09-07   
...             ...               ...             ...       ...         ...   
674433          Sat     unlabelled_18            1411      1965  2022-04-02   
674434          Sat     unlabelled_18             200      1965  2022-04-02   
674435          Sat     unlabelled_18             524      1965  2022-04-02   
674436          Sat     unlabelled_18            1382      1965  2022-04-02   
674437          Sat     unlabelled_18             795      1965  2022-04-02   

        Lengte_vaartu

In [11]:
data = dd.dataset.copy()
data.shape

(674438, 14)

In [90]:
# Custom outlier detection functions:
def find_anomalies(value, lower_threshold, upper_threshold):
    if value < lower_threshold or value > upper_threshold:
        return 1
    else: return 0
def iqr_anomaly_detector(data, column, threshold=1.5):
    df = data.copy()
    quartiles = dict(data[column].quantile([.25, .50, .75]))
    quartile_3, quartile_1 = quartiles[0.75], quartiles[0.25]
    iqr = quartile_3 - quartile_1
    lower_threshold = quartile_1 - (threshold * iqr)
    upper_threshold = quartile_3 + (threshold * iqr)
    df['Predictions'] = data[column].apply(find_anomalies, args=(lower_threshold, upper_threshold))
    return df

In [94]:
def pout(x):
    features = list(x.select_dtypes('number').columns)
    features.remove('dataset_index') # generiek genoeg? wel nodig, want anders blijft dataset_index ondanks de groupby er alsnog tussen staan, en dan werkt reset_index() niet
    
    result = { fi : np.unique(iqr_anomaly_detector(x,fi).Predictions,return_counts=True)[1][1]/len(x[fi])*100 for fi in features }
    return pd.Series(result)

In [100]:
tmp = data.loc[data['dataset_index']==10]

In [91]:
fi = features[0]
counts = np.unique(iqr_anomaly_detector(tmp,fi).Predictions,return_counts=True)[1][1]/len(tmp[fi])*100

In [84]:
counts

14.189993631807734

In [101]:
outl = pout(tmp)

In [102]:
outl

Breedte_vaartuig     49.983270
Lengte_vaartuig      49.972117
Bouwjaar             47.978474
Lon_insploc          13.615325
Tonnage              49.947022
Lat_insploc          13.615325
Diepgang_vaartuig    49.431184
laatstAlgemeen       24.988847
dtype: float64

In [105]:
data = data.groupby(['dataset_index', 'dataset_date']).apply(pout)

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
outl = []
for tx,ti in enumerate(tix):
    for fi in feats:
        iqr_df = iqr_anomaly_detector(ds.loc[ds['datT']==ti],fi)
        unique, counts = np.unique(iqr_df.Predictions, return_counts=True)
        if len(counts)>1:
            OUT = counts[1]/len(iqr_df.Predictions)*100
        else:
            OUT = 0
        outl.append(
            {
                'Time': tid[tx],
                'Feature': fi,
                'poutliers': OUT
            }
        )

In [16]:
data = dd.dataset

# uitlezen gevallen waar de markering 'train' is of begint met 'labelled'
train_mask = data.dataset_markering == 'train'
labelled_mask = data.dataset_markering.str.contains('^labelled')
mask = train_mask | labelled_mask # samenstellen mask
labelled_data = data[mask] # subsamplen van labelled data

# uitlezen gevallen waar het om out of sample gaat ('oos' of 'unlabelled')
oos_mask = data.dataset_markering == 'oos'   # dataset maken met alleen out of sample data
unlabelled = data.dataset_markering.str.contains('^unlabelled')  # out of sample data in de drift datasets begint met 'unlabelled'
mask = oos_mask | unlabelled  # samenstellen mask
oos = data[mask] # subsamplen van out of sample data


In [19]:
df = oos.groupby(['dataset_index', 'dataset_date']).count()   # this results in a dataframe with MultiIndex, which plotly doesn't understand. 

In [20]:
df

,,insp_weekdag,dataset_markering,laatstAlgemeen,Bouwjaar,Begindatum,Lengte_vaartuig,soortBeroepsvaartuig,Lon_insploc,Lat_insploc,Breedte_vaartuig,Tonnage,Diepgang_vaartuig
dataset_index,dataset_date,,,,,,,,,,,,
0,2020-03-16,31307,31307,31307,31307,31307,31307,31307,31307,31307,31307,31307,31307
1,2021-02-01,30705,30705,30705,30705,30705,30705,30705,30705,30705,30705,30705,30705
2,2021-09-28,30992,30992,30992,30992,30992,30992,30992,30992,30992,30992,30992,30992
3,2021-10-06,31008,31008,31008,31008,31008,31008,31008,31008,31008,31008,31008,31008
4,2021-10-28,31014,31014,31014,31014,31014,31014,31014,31014,31014,31014,31014,31014
5,2021-11-10,30960,30960,30960,30960,30960,30960,30960,30960,30960,30960,30960,30960
6,2021-11-10,30960,30960,30960,30960,30960,30960,30960,30960,30960,30960,30960,30960
7,2021-11-15,30962,30962,30962,30962,30962,30962,30962,30962,30962,30962,30962,30962
8,2021-12-01,30970,30970,30970,30970,30970,30970,30970,30970,30970,30970,30970,30970
